# Deploy MLflow Model to online endpoint

- Before running this notebook, run the **Chapter 6 Prep-Model Creation & Registration.ipynb** notebook to create and register a model for use

- Models created with MLFlow do not require a scoring script nor an environment

## In this notebook we will:

- Connect to your workspace.
- Create an online endpoint
- Retrieve and register a model from the job ran in the previous notebook
- Create a deployment
- Make an API Call to the managed online endpoint

You can use either the Python 3.10 - SDK V2 kernel, or your job_env kenel to run this notebook.

**Kernel** > **Change Kernel** > **Python 3.10 - SDK V2**

or if you already setup the virtual environment in Chapter 4:

Select **Kernel** > **Change Kernel** > **job_env**

In [1]:
import azure.ai.ml
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)

print(azure.ai.ml._version.VERSION)

1.1.0


In [2]:
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AML_WORKSPACE_NAME>"

In [3]:
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

## Create Online Endpoint

To create an online endpoint, we will leverage the class *ManagedOnlineEndpoint*.  To create the online endpoint we will provide the following configuration:

- name of endpoint
- description
- auth_mode (set to key) or aml_token
- tags - to provide information regarding the endpoint

### Configure endpoint

In [4]:
# Creating a unique endpoint name with current datetime to avoid conflicts
import datetime

online_endpoint_name = "chp6-mlflow-endpt-" + datetime.datetime.now().strftime("%m%d%H%M%f")
# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="titanic online endpoint for mlflow model",
    auth_mode="key",
    tags={"mlflow": "true"},
)

## Create endpoint

Using the MLClient created earlier, we will now create the Endpoint in the workspace. This command will start the endpoint creation and return a confirmation response while the endpoint creation continues.


In [5]:
ml_client.begin_create_or_update(endpoint)

## Create deployment
A deployment is a set of resouces used for hosting the inferecing model using the *ManagedOnlineDeployment* class.  
Using the *ManagedOnlineDeployment* class, a developer can configure the following components

- name: name of the deployment
- endpoint_name: name of the endpoint to create the deployment under
- model: the model to use for the deployment
- instance_type: the VM side to use for deployment
- instance_count: the number of instances to use for the deployment
    

# Retrieve Model from Workspace

In [6]:
run_model = ml_client.models.get(name="chapter6_titanic_model", version="1")

## Configure the deployment

- retrieve the experiment id for this run, and the run id to retrieve the model from the registered model list

In [7]:
import time
while ml_client.online_endpoints.get(name=online_endpoint_name).provisioning_state == 'Creating':
    print('Creating')
    time.sleep(15)

print(ml_client.online_endpoints.get(name=online_endpoint_name).provisioning_state)

Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Succeeded


In [8]:
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=run_model,
    instance_type="Standard_F4s_v2",
    instance_count=1,
)

In [9]:
## Create deployment
print(online_endpoint_name)

chp6-mlflow-endpt-12102106054771


In [10]:
ml_client.online_deployments.begin_create_or_update(blue_deployment)

Check: endpoint chp6-mlflow-endpt-12102106054771 exists
data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored


In [11]:
import time
while ml_client.online_deployments.get(name = "blue", endpoint_name = online_endpoint_name).provisioning_state == 'Updating':
    print('Updating..will take about 10 minutes to deploy...')
    time.sleep(15)
    
ml_client.online_deployments.get(name = "blue", endpoint_name = online_endpoint_name).provisioning_state

.Updating..will take about 10 minutes to deploy...
..Updating..will take about 10 minutes to deploy...
...Updating..will take about 10 minutes to deploy...
...Updating..will take about 10 minutes to deploy...
...Updating..will take about 10 minutes to deploy...
...Updating..will take about 10 minutes to deploy...
...Updating..will take about 10 minutes to deploy...
...Updating..will take about 10 minutes to deploy...
...Updating..will take about 10 minutes to deploy...
...Updating..will take about 10 minutes to deploy...
...Updating..will take about 10 minutes to deploy...
...Updating..will take about 10 minutes to deploy...
...Updating..will take about 10 minutes to deploy...
...Updating..will take about 10 minutes to deploy...
...Updating..will take about 10 minutes to deploy...
...Updating..will take about 10 minutes to deploy...
...Updating..will take about 10 minutes to deploy...
...Updating..will take about 10 minutes to deploy...
...Updating..will take about 10 minutes to deploy

'Succeeded'

In [12]:
## get status of online deployment
ml_client.online_deployments.get_logs(
    name="blue", endpoint_name=online_endpoint_name, lines=50
)

"Instance status:\nSystemSetup: Succeeded\nUserContainerImagePull: Succeeded\nModelDownload: Succeeded\nUserContainerStart: Succeeded\n\nContainer events:\nKind: Pod, Name: Started, Type: Normal, Time: 2022-12-10T21:19:14.635778Z, Message: Started container inference-server\nKind: Pod, Name: ReadinessProbeFailed, Type: Warning, Time: 2022-12-10T21:19:30.648767Z, Message: Readiness probe failed: HTTP probe failed with statuscode: 502\nKind: Pod, Name: LivenessProbeFailed, Type: Warning, Time: 2022-12-10T21:19:32.372952Z, Message: Liveness probe failed: HTTP probe failed with statuscode: 502\nKind: Pod, Name: ReadinessProbeFailed, Type: Warning, Time: 2022-12-10T21:19:40.648584Z, Message: Readiness probe failed: HTTP probe failed with statuscode: 502\nKind: Pod, Name: LivenessProbeFailed, Type: Warning, Time: 2022-12-10T21:19:42.373102Z, Message: Liveness probe failed: HTTP probe failed with statuscode: 502\nKind: Pod, Name: ReadinessProbeFailed, Type: Warning, Time: 2022-12-10T21:19:50.

In [13]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
ml_client.begin_create_or_update(endpoint)

## Checking provisioning state

In [14]:
import time
while ml_client.online_endpoints.get(name=online_endpoint_name).provisioning_state == 'Updating':
    print('Updating')
    time.sleep(15)
    
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)
endpoint.provisioning_state

Updating
Updating


'Succeeded'

## Get Endpoint details

In [15]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(endpoint)
print(' ')
# existing traffic details
print(endpoint.traffic)
print(' ')
# Get the scoring URI
print('uri: ' + str(endpoint.scoring_uri))
primary_key = ml_client.online_endpoints.get_keys(name = online_endpoint_name).primary_key
print(' ')
print('primary key: ' + str(primary_key))

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://chp6-mlflow-endpt-12102106054771.eastus.inference.ml.azure.com/score', 'openapi_uri': 'https://chp6-mlflow-endpt-12102106054771.eastus.inference.ml.azure.com/swagger.json', 'name': 'chp6-mlflow-endpt-12102106054771', 'description': 'titanic online endpoint for mlflow model', 'tags': {'mlflow': 'true'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/5da07161-3770-4a4b-aa43-418cbbb627cf/resourcegroups/chapter6-azureml-rg/providers/microsoft.machinelearningservices/workspaces/chapter6-azureml/onlineendpoints/chp6-mlflow-endpt-12102106054771', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/5da07161-3770-4a4b-aa43-418cbbb627cf/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/oe:9e44248b-9077-4922-99ad-0c577c53684e:e7497d00-9689-453e-935f-f45adc840b94?api-version=2022-02-01-preview'}, 'id': '/subscriptions/5da

### Leverage the registered ml table from Chapter 4 to get some data to send to the rest endpoint.

- Previously in Chapter 4, we registered the MLTable: titanic_prepped_mltable_x2, we will retrieve it, if we don't have it registered, we can leverage the data directly

In [16]:
import mltable
import pandas as pd

try:
    registered_v1_data_asset = ml_client.data.get(name='titanic_prepped_mltable_x2', version='1')
    print(registered_v1_data_asset.path)

    tbl = mltable.load(uri=registered_v1_data_asset.path)
    df = tbl.to_pandas_dataframe()
    print('retrieved data frame from registered mltable')
except:
   
    df = pd.read_csv('./prepped_data/titanic_prepped.csv')
    
columns_to_keep =  ['Embarked', 'Loc', 'Sex','Pclass', 'Age', 'Fare', 'GroupSize']
X_raw           = df[columns_to_keep].head(5)
X_raw

azureml://subscriptions/5da07161-3770-4a4b-aa43-418cbbb627cf/resourcegroups/chapter6-azureml-rg/workspaces/chapter6-azureml/datastores/workspaceblobstore/paths/LocalUpload/b3e9d2d76d36b52fc88b17546f0f0460/titanic_prepped_mltable/


Unrecognized authentication type: None
Unrecognized authentication type: None


retrieved data frame from registered mltable


,Embarked,Loc,Sex,Pclass,Age,Fare,GroupSize
0,S,X,m,3,22.0,7.25,2
1,C,C,f,1,38.0,71.2833,2
2,S,X,f,3,26.0,7.925,1
3,S,C,f,1,35.0,53.1,2
4,S,X,m,3,35.0,8.05,1


In [17]:
columns_to_keep =  ['Embarked', 'Loc', 'Sex','Pclass', 'Age', 'Fare', 'GroupSize']
X_raw           = df[columns_to_keep]
X_raw

,Embarked,Loc,Sex,Pclass,Age,Fare,GroupSize
0,S,X,m,3,22.0,7.25,2
1,C,C,f,1,38.0,71.2833,2
2,S,X,f,3,26.0,7.925,1
3,S,C,f,1,35.0,53.1,2
4,S,X,m,3,35.0,8.05,1
...,...,...,...,...,...,...,...
886,S,X,m,2,27.0,13.0,1
887,S,B,f,1,19.0,30.0,1
888,S,X,f,3,21.5,23.45,4
889,C,C,m,1,26.0,30.0,1


In [18]:
import json
url = endpoint.scoring_uri
api_key = primary_key  # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'blue' }
import requests

def make_prediction(df):
    strjson = str(df.to_json(orient='records'))
    endpoint_url = url

    request_data =  {
              "input_data": {
                "columns": [
                  "Embarked",
                  "Loc",
                  "Sex",
                  "Pclass",
                  "Age",
                  "Fare",
                  "GroupSize"
                ],
                "data": []
              }
            }

    request_df = X_raw.head(2)
    request_data['input_data']['data'] = json.loads(request_df.to_json(orient='records'))
    parsed = json.dumps(request_data)
    print(parsed)
    r = requests.post(endpoint_url, headers=headers, data=parsed)
    return (r.json())

results = make_prediction(X_raw.head(2))
print('')
print('predictions')
print(results)

{"input_data": {"columns": ["Embarked", "Loc", "Sex", "Pclass", "Age", "Fare", "GroupSize"], "data": [{"Embarked": "S", "Loc": "X", "Sex": "m", "Pclass": "3", "Age": "22.0", "Fare": "7.25", "GroupSize": "2"}, {"Embarked": "C", "Loc": "C", "Sex": "f", "Pclass": "1", "Age": "38.0", "Fare": "71.2833", "GroupSize": "2"}]}}

predictions
[0, 1]
